In [1]:
!pip install rdflib
!pip install folium
!pip install pyproj

     |████████████████████████████████| 407 kB 7.6 MB/s 
     |████████████████████████████████| 45 kB 2.1 MB/s 
     |████████████████████████████████| 6.3 MB 7.3 MB/s 


In [2]:
storage = "https://raw.githubusercontent.com/opencitydata/guia-rdf-datosgob/main/rdf-explotacion/terrazas-madrid.nt" #poner el enlace a tus datos en github (raw files)

from rdflib import Graph, Namespace, Literal
from rdflib.plugins.sparql import prepareQuery
import folium
from pyproj import Transformer

g = Graph()

g.parse(storage, format="ntriples") #quizá esto tarde un poco si el archivo es muy grande

<Graph identifier=Nc3841ddcc6094124a19a111033a8d810 (<class 'rdflib.graph.Graph'>)>

# Consulta 1: Listado de terrazas y sus horarios de lunes a viernes anualmente

In [ ]:
from rdflib import XSD

ESCOM = Namespace("http://vocab.ciudadesabiertas.es/def/comercio/tejido-comercial/") 

q1 = prepareQuery('''
  SELECT 
    ?terraza ?horario
	WHERE { 
    ?terraza rdf:type escom:Terraza;
          <http://schema.org/openingHours> ?horario .
    
    FILTER(regex(?horario, "Anual Lun-Juev.*", "i" ))
  } 
  ''',
  initNs = { "escom": ESCOM}
)

for r in g.query(q1):
  print(r.terraza, r.horario)

# Consulta 2: Listado de terrazas que tengan más de 15 mesas autorizadas

In [ ]:
from rdflib import XSD

ESCOM = Namespace("http://vocab.ciudadesabiertas.es/def/comercio/tejido-comercial/") 

q1 = prepareQuery('''
  SELECT 
    ?terraza ?mesas
	WHERE { 
    ?terraza rdf:type escom:Terraza;
          escom:numeroMesasAutorizadas ?mesas .
    
    FILTER(?mesas > "15"^^<http://www.w3.org/2001/XMLSchema#integer>)
  } 
  ''',
  initNs = { "escom": ESCOM}
)

for r in g.query(q1):
  print(r.terraza, r.mesas)


# Consulta 3: Listado de terrazas con actividad en el periodo anual

In [ ]:
from rdflib import XSD

ESCOM = Namespace("http://vocab.ciudadesabiertas.es/def/comercio/tejido-comercial/") 
q1 = prepareQuery('''
  SELECT 
    ?terraza
	WHERE { 
    ?terraza rdf:type escom:Terraza;
          escom:periodoFuncionamiento <http://vocab.linkeddata.es/datosabiertos/kos/comercio/periodo-funcionamiento/anual> .
  } 
  ''',
  initNs = { "escom": ESCOM}
)

for r in g.query(q1):
  print(r.terraza)

# Ejemplo real: Dibujando puntos geográficos a partir de RDF

In [5]:
ESCOM = Namespace("http://vocab.ciudadesabiertas.es/def/comercio/tejido-comercial/") 
# Preparamos la consulta, dame las terrazas, su horario anual de lunes a jueves y la latitud y longitud de su LocalComercial asociado
q1 = prepareQuery('''
  SELECT 
    ?horario ?lat ?lon
	WHERE { 
    ?terraza rdf:type escom:Terraza .
    ?terraza <http://schema.org/openingHours> ?horario .
    ?terraza escom:perteneceA ?local .
    ?local rdf:type escom:LocalComercial .
    ?local <http://www.opengis.net/ont/geosparql#hasGeometry> ?point .
    ?point rdf:type <http://www.opengis.net/ont/sf#Point> .
    ?point <https://datos.ign.es/def/geo_core#xETRS89> ?lat .
    ?point <https://datos.ign.es/def/geo_core#yETRS89> ?lon .
    
    FILTER(regex(?horario, "Anual Lun-Juev.*", "i" ))
  } LIMIT 10
  ''',
  initNs = { "escom": ESCOM}
)
# inspeccionamos los datos que nos devuelve la consulta
for r in g.query(q1):
  print(r.lat, r.lon, r.horario)

# debemos transformar el formato de los datos de lon de UTM a WGS 84
transformer = Transformer.from_crs('epsg:25830','epsg:4326')
mapa = folium.Map(location=[40.4167,  -3.70325])
for r in g.query(q1):
  x,y = transformer.transform(float((r.lat).replace(",",".")),float((r.lon).replace(",",".")))
  horario = (r.horario).replace("Anual Lun-Juev ","")
  folium.Marker([x,y], popup=horario, tooltip=horario).add_to(mapa)

mapa

443905,56 4471544,48 Anual Lun-Juev 10:00:00 - 00:00:00
440777,59 4474441,52 Anual Lun-Juev 10:00:00 - 00:00:00
441732,6 4478748,52 Anual Lun-Juev 10:00:00 - 00:00:00
446935,57 4476568,45 Anual Lun-Juev 08:00:00 - 01:00:00
439161,6 4475784,54 Anual Lun-Juev 10:00:00 - 00:00:00
442577,58 4473167,5 Anual Lun-Juev 10:00:00 - 00:00:00
440603,59 4473300,52 Anual Lun-Juev 10:00:00 - 23:00:00
435267,59 4468465,56 Anual Lun-Juev 10:00:00 - 00:00:00
443015,59 4477650,5 Anual Lun-Juev 10:00:00 - 00:00:00
434793,63 4471955,57 Anual Lun-Juev 10:00:00 - 00:00:00
